# Introduction to Natural Language Processing
# Name: Sai Krishna Lakshminarayanan Student ID: 18230229
# Assignment 1

This assignment will involve the creation of a spellchecking system and an evaluation of its performance. You may use the code snippets provided in Python for completing this or you may use the programming language or environment of your choice

Please start by downloading the corpus `holbrook.txt` from Blackboard

The file consists of lines of text, with one sentence per line. Errors in the line are marked with a `|` as follows

    My siter|sister go|goes to Tonbury .
    
In this case the word 'siter' was corrected to 'sister' and the word 'go' was corrected to 'goes'.

In some places in the corpus two words maybe corrected to a single word or one word to a multiple words. This is denoted in the data using underscores e.g.,

    My Mum goes out some_times|sometimes .
    
For the purpose of this assignment you do not need to separate these words, but instead you may treat them like a single token.

*Note: you may use any functions from NLTK to complete the assignment. It should not be necessary to use other libraries and so please consult with us if your solution involves any other external library. If you use any function from NLTK in Task 6 please include a brief description of this function and how it contributes to your solution.*

## Task 1 (10 Marks)

Write a parser that can read all the lines of the file `holbrook.txt` and print out for each line the original (misspelled) text, the corrected text and the indexes of any changes. The indexes refers to the index of the words in the sentence. In the example given, there is only an error in the 10th word and so the list of indexes is [9]. It is not necessary to analyze where the error occurs inside the word.

Then split your data into a test set of 100 lines and a training set.

In [26]:
import nltk #importing the required pacakages
from nltk.util import ngrams
from nltk.metrics.distance import edit_distance
from nltk.corpus import words
from nltk.tokenize import word_tokenize

In [27]:
def parsing(sent):  #defining parsing function to split the given sentence
    original_line = [] #this is going to be the original line
    corrected_line = [] #this will be the corrected line
    indexes = [] #index of the word error in the given sentence
    count = 0 
    
    for i in sent:
        if '|' in i:#when there is error
            str1 = i.split('|')# Splitting the sentence based on '|'
            original_line.append(str1[0])# Previous word to '|' 
            corrected_line.append(str1[1])# Next word to '|' 
            indexes.append(count)# index of error
        
        else:#when no error
            original_line.append(i)
            corrected_line.append(i)
        count = count+1   
    final = {'original': original_line, 'corrected': corrected_line, 'indexes': indexes}
    
    return final


In [28]:
def preprocess(): # to perform perprocess to get data in an executable format
    data = []
    text_file = open("holbrook.txt", "r")#reading the given text file
    lines = []
    for i in text_file:
        lines.append(i.strip())
    sentences = [nltk.word_tokenize(sent) for sent in lines]#word tokenization of the given sentence
    for sent in sentences:
        data.append(parsing(sent))# parse the given sentence and to correct it 
    
    return data

data = preprocess()#executing the given text file data

The counts and assertions given in the following sections are based on splitting the training and test set as follows

In [29]:
#performing cross check as given in the question
print(data[2])
assert(data[2] == {
 'original': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'siter', '.'],
 'corrected': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'sister', '.'],
 'indexes': [9]
})

# Splitting the data as given in the question
test = data[:100]
train = data[100:]

{'corrected': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'sister', '.'], 'original': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'siter', '.'], 'indexes': [9]}


## **Task 2** (10 Marks): 
Calculate the frequency (number of occurrences), *ignoring case*, of all words and their unigram probability from the corrected *training* sentences.

*Hint: use `Counter` to implement this so it may be called many times*

In [30]:
corrected_train = [elem['corrected'] for elem in train]# taking the values of corrected alone from the train
original_test = [elem['original'] for elem in test]#taking the values of original alone from the test
corrected_test = [elem['corrected'] for elem in test] #taking the values of corrected alone from the test
from collections import Counter
def unigram(word):#Unigram function to return the frequency of given word
    a = []
    word = word.lower()#lower casing the given word
    for i in corrected_train:#checking each line of corrected train
        a.append(" ".join(i).lower())#lower casing it
    a = " ".join(a)    
    a = nltk.word_tokenize(a)# word tokenization on a
    unigram_frquency = nltk.FreqDist(a) # getting the unigram frquency
    return unigram_frquency[word]
print("unigram('me')==", unigram('me')) #desired result
assert unigram('me')==87 #cross verification of the unigram function

("unigram('me')==", 87)


In [31]:
def prob(word): # to obtain the probability of the unigram word
    b = unigram(word) #getting the frequency of the given word
    a=0
    for i in corrected_train:
        a=a+len(i)#total number of words present in the given corrected train
    
    probability=float(b)/float(a)#required unigram probability
    return probability
print(prob('me'))#desired result

0.0039891787794


## **Task 3** (15 Marks): 
[Edit distance](https://en.wikipedia.org/wiki/Edit_distance) is a method that calculates how similar two strings are to one another by counting the minimum number of operations required to transform one string into the other. There is a built-in implementation in NLTK that works as follows:


In [32]:
from nltk.metrics.distance import edit_distance

# Edit distance returns the number of changes to transform one word to another
print(edit_distance("hello", "hi"))

4


Write a function that calculates all words with *minimal* edit distance to the misspelled word. You should do this as follows

1. Collect the set of all unique tokens in `train`
2. Find the minimal edit distance, that is the lowest value for the function `edit_distance` between `token` and a word in `train`
3. Output all unique words in `train` that have this same (minimal) `edit_distance` value

*Do not implement edit distance, use the built-in NLTK function `edit_distance`*

In [33]:
def get_candidates(token):# to replace the given misspelled word through suggestion by means of edit distance
    
    a = []
    
    for i in corrected_train:
        a.append(" ".join(i).lower())#lower casing all the words in corrected_train

    a = " ".join(a)
    a = nltk.word_tokenize(a)
    unigram_frequency = nltk.FreqDist(a)#obtaining the frequency of the words in the given list
    unique_words = list(unigram_frequency.keys()) # getting each unique word corresponding to it by help of keys

   
    s = []
    for i in unique_words:# checking every unique word
        if edit_distance(i, token)==1 : #when the distance between the given misspelled word and suggestion is 1
            s.append("".join(i))
    
  
    return s #desired output
print(get_candidates("minde"))
assert get_candidates("minde") == ['mind', 'mine'] # cross verification of the get_candidate function

['mind', 'mine']


## Task 4 (15 Marks):

Write a function that takes a (misspelled) sentence and returns the corrected version of that sentence. The system should scan the sentence for words that are not in the dictionary (set of unique words in the training set) and for each word that is not in the dictionary choose a word in the dictionary that has minimal edit distance and has the highest *unigram probability*. 

*Your solution to this should involve `get_candidates`*


In [34]:
a = []

for i in corrected_train: #checking every line of the given corrected train
    a.append(" ".join(i).lower())#lower casing all of them

a = " ".join(a)
a = nltk.word_tokenize(a)# word tokenization of the corrected train
unigram_frequency = nltk.FreqDist(a) # obtaining the unigram frequency of the words in corrected train
unique_words = list(unigram_frequency.keys()) # list of unique words present in the corrected train
p=[] # to store unigram probability

def correct(sentence): # function to correct the given sentence 
    corrected = [] #to store the corrected sentence
    count = 0
    indexes = [] #to show the index of the misspelled word to get the suggestions
    for i in sentence: #checking every word in the given sentence
        if i.lower() not in unique_words:# when the given word is not present in the unique words list of corrected train
            indexes.append(count)
            if len(get_candidates(i)) > 1:# when there are two or more suggestions available to a given misspelled word

                suggestion = get_candidates(i)#obtaining the suggestions
                p.append(prob(i))# getting the unigram probability for all the suggested words
                corrected.append(suggestion[p.index(max(p))])#returning the suggested word as the one with maximum probability
            else:
                corrected.append("".join(get_candidates(i)))# directly obtain the suggestion for the misspelled word 

        else:
            corrected.append(i)# when no correction is needed
        count = count+1
    return corrected #returning the corrected sentence


print(correct(["this","whitr","cat"]))#desired output
assert(correct(["this","whitr","cat"]) == ['this','white','cat'])#cross verification of the correct function

['this', 'white', 'cat']


## **Task 5** (10 Marks): 
Using the test corpus evaluate the *accuracy* of your method, i.e., how many words from your system's output match the corrected sentence (you should count words that are already spelled correctly and not changed by the system).

In [35]:
def accuracy(test):# to determine the accuracy of the algorithm  
    correctedSentences = []
    ans=0
    for line in original_test:#checking every line of the original test
        cor = correct(line)# correcting the wrong ones using the correct function
        correctedSentences.append(cor) #storing it inthe correctedSentences list
    accuracy = []# to store the accuracy of the mechanism
    for i in range(len(corrected_test)):#to perform it till the length of the corrected test
        count = 0
        for index in range(len(corrected_test[i])):
            if(correctedSentences[i][index].lower() == corrected_test[i][index].lower()):
                #when sentence corrected by correct function and corrected sentence in the test are equal
                count += 1    
        accuracy.append(float(count)/len(corrected_test[i]))#storing the accuracy for each instances
    ans=float(sum(accuracy)/len(accuracy))*100#calculating the overall accuracy for test data
    return (ans)

print "The accuracy % of the algorithm for the test data is  ",accuracy(test)

The accuracy % of the algorithm for the test data is   82.653182339


## **Task 6 (35 Marks):**

Consider a modification to your algorithm that would improve the accuracy of the algorithm developed in Task 3 and 4

* You may resources beyond those provided here.
* You must **not use the test data** in this task.
* Provide a short text describing what you intend to do and why. 
* Full marks for this section may be obtained without an implementation, but an implementation is preferred.
* Your implementation should not consist of more than 50 lines of code

Please note this task is marked according to: demonstration of knowledge from the lecutures (10), originality and appropriateness of solution (10), completeness of description (10) and technical correctness (5)


The algorithm for sentence detection can be improved in various ways. Here , we considered only unigram probability and did suggestion and correction based on that. But this can be improved inorder to get better accuracy and an enhanced algorithm.Now let us check the ways in which it can be helped to imrpoved

here are various ways to improve the accuracy of the algorithm like tense detection, Named entity detection, Bigrams, trigrams, smoothing , Noise Free Corpus etc .  A few methods are described below : 

1 Checking the kind of error user makes[implemented]

*  The user sometimes forgets to provide strokes properly and there occurs punctuation errors. They donot help in improving the efficiency of the code
*  These punctuations can be simply ignored inorder to get better accuracy for the algorithm

2 Bigram[implemented]

* Unigram is implemented in the algorithm. When it is known that 2 words come together usually then it is advisable to perform bigram frequency on it and find its probability to suggest words which will help in executing two words in place of one at a time.This helps in increasing the accuracy of the algorithm.

3 Trigram

* Trigram probability can also be implemented . This will come in handy when the given sentence is very long and unigram results in consumption of alot of time. So, trigram will help in checking three words at a time and there by reducing the time and as a result increasing the efficiency of the algorithm.


4 Part of Speech Tag

* Part of Speech Tag provides tagging of each word in the given data.
* This helps in training the data set better inorder to understand each word based on its position and whether it is a noun or a verb etc.
* This coupled up in suggestion function helps in giving out better yielding suggested words to the misspelled words thereby increasing the accuracy of the algorithm

5  Inverse Document Frequency

* There are some words that occur frequently in sentences or in data like the stopwords.
* When the term frequencies for such words are down scaled, it helps in the betterment of the performance
* This inverse document frequency helps in determining how important a word is in a given data. This helps us to ignore the least important ones which will help in increasing the accuracy.

6 Large Stopword list 

* Stopwords hold no value in sentence processing.
* So, analysing them and having an elaborate list of stopwords like a,the etc will help us in creating a function to ignore them when they occur.
* This reduces the words to be processed thereby increasing the accuracy

7 Ignoring least Frequency Words

* There will be inert words in any given data will occur only once or so.
* They dont help in improving the training the data and just results in increase of the count.
* When these words are excluded, the count gets reduced thereby increasing the accuracy

8  Name Entity Recognition

* The algorithm may not be able to understand a name directly.
* Therefore, it is important to train the data with names of the people and objects so that it doesn't consider them to be any junk value and gives out proper result.
* This process is very important and improves the accuracy of the algorithm greatly.

9 Lemmatization

* A word occurs in various forms and tenses in the given data.
* This makes the training to be provided in such a way that each word occuring in various forms and tenses to be different.
* Therefore, when lemmatization is done, it condenses them to their base word format.
* From the base word, it is easy to get the suggestion out for the given one.
* This helps in providing training in an efficient way with reduced count of words to be considered.
* This increases the accuracy of the algorithm greatly.

10  Smoothing

* Smoothing can be done to understand the functioning of the algorithm in visual manner also.
* It can be done in several ways so that probability is not null whenever execution is made.







In [36]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
corrected_train = [elem['corrected'] for elem in test]
original_test = [elem['original'] for elem in test]
    
    # Removing all special characters from the list.
original_test = [tokenizer.tokenize(" ".join(elem)) for elem in original_test]
corrected_train = [tokenizer.tokenize(" ".join(elem)) for elem in corrected_train]
corrected_train = [tokenizer.tokenize(" ".join(elem)) for elem in corrected_train]

In [49]:
def bigram(words): #function to get the bigram frequency of the given words
    a = []
    
    # This function get words in string, hence converting string of 2 words to tuple.
    words = words.split(" ")#splitting the words
    words[0] = words[0].lower()#lower casing the first word
    words[1] = words[1].lower()#lower casing the second word
    words = tuple(words) #creating tuple so that it cannot be changed
    for i in corrected_train: # checking every line in corrected train
        a.append(" ".join(i))
        
    a = " ".join(a)    
    a = a.lower()#lower casing them
    
    #Calculating Bigrams for given words.
    tokens = nltk.wordpunct_tokenize(a) #tokenizing a based on wordpunct
    bigrams=nltk.collocations.BigramCollocationFinder.from_words(tokens)#getting the bigrams from collacation finder
    bigram_frequency = dict(bigrams.ngram_fd)#obtaining the bigram frquency of the words
    
    return bigram_frequency[words]

print "bigram('you are')==",bigram("you are")#desired output
assert bigram("you are")==1

bigram('you are')== 1


## **Task 7 (5 Marks):**

Repeat the evaluation (as in Task 5) of your new algorithm and show that it outperforms the algorithm from Task 3 and 4

On repeating the task 5 for the changes implemented as in task 6, it is seen that accuracy as improved to an extent thereby increasing the efficiency of the modified alogirithm.

In [ ]:
def accuracy(test):# to determine the accuracy of the algorithm  
    correctedSentences = []
    ans=0
    for line in original_test:#checking every line of the original test
        cor = correct(line)# correcting the wrong ones using the correct function
        correctedSentences.append(cor) #storing it inthe correctedSentences list
    accuracy = []# to store the accuracy of the mechanism
    for i in range(len(corrected_test)):#to perform it till the length of the corrected test
        count = 0
        for index in range(len(corrected_test[i])):
            if(correctedSentences[i][index].lower() == corrected_test[i][index].lower()):
                #when sentence corrected by correct function and corrected sentence in the test are equal
                count += 1    
        accuracy.append(float(count)/len(corrected_test[i]))#storing the accuracy for each instances
    ans=float(sum(accuracy)/len(accuracy))*100#calculating the overall accuracy for test data
    return (ans)

print "The accuracy % of the algorithm for the test data is  ",accuracy(test)